In [16]:
!rm -rf /content/2018_data_clean/

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [4]:
import findspark
findspark.init()
spark_url = 'local'

import os
import json
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

In [29]:
import zipfile
import os

# 1. แตกไฟล์ ZIP
input_path = "/content/2023.zip"
output_folder = "/content/2023_extracted"

os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print(f"Files extracted to: {output_folder}")

Files extracted to: /content/2023_extracted


In [31]:
# Paths
folder_path = r"/content/2023_extracted"
output_csv = r"/content/2023_coredata"

# Function to process each file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as json_file:
        try:
            data = json.load(json_file)

            # Ensure the JSON is a dictionary
            if not isinstance(data, dict):
                print(f"Unexpected structure in file: {file_path}")
                return [Row(filename=os.path.basename(file_path), status="Invalid JSON structure")]

            # Extract coredata, affiliations, and subject-areas
            coredata = data.get('abstracts-retrieval-response', {}).get('coredata', {})
            affiliations = data.get('abstracts-retrieval-response', {}).get('affiliation', [])
            subject_areas = data.get('abstracts-retrieval-response', {}).get('subject-areas', {}).get('subject-area', [])

            # Handle cases where affiliation is not a list
            if isinstance(affiliations, dict):  # If it's a single dictionary, convert it to a list
                affiliations = [affiliations]
            elif not isinstance(affiliations, list):  # Ensure it's a list
                affiliations = []

            # Ensure subject-areas is a list
            if not isinstance(subject_areas, list):
                subject_areas = []

            rows = []
            for affiliation in affiliations:
                # Ensure the affiliation is a dictionary
                if not isinstance(affiliation, dict):
                    continue

                for subject in subject_areas:
                    # Ensure the subject is a dictionary
                    if not isinstance(subject, dict):
                        continue

                    row = Row(
                        filename=os.path.basename(file_path),
                        status="Processed",
                        srctype=coredata.get('srctype', None),
                        eid=coredata.get('eid', None),
                        prism_coverDate=coredata.get('prism:coverDate', None),
                        prism_aggregationType=coredata.get('prism:aggregationType', None),
                        subtypeDescription=coredata.get('subtypeDescription', None),
                        affiliation_city=affiliation.get('affiliation-city', None),
                        affiliation_id=affiliation.get('@id', None),
                        affiliation_name=affiliation.get('affilname', None),
                        affiliation_country=affiliation.get('affiliation-country', None),
                        subject_area_name=subject.get('$', None),
                        subject_area_code=subject.get('@code', None),
                        subject_area_abbrev=subject.get('@abbrev', None),
                        publicationName=coredata.get('prism:publicationName', None),
                        source_id=coredata.get('source-id', None),
                        citedby_count=coredata.get('citedby-count', None),
                        subtype=coredata.get('subtype', None),
                        dc_title=coredata.get('dc:title', None),
                        prism_doi=coredata.get('prism:doi', None),
                        dc_identifier=coredata.get('dc:identifier', None),
                        dc_publisher=coredata.get('dc:publisher', None)
                    )
                    rows.append(row)

            if not rows:  # No valid rows, return a default
                rows.append(Row(
                    filename=os.path.basename(file_path),
                    status="No valid data",
                    srctype=None, eid=None, prism_coverDate=None,
                    prism_aggregationType=None, subtypeDescription=None,
                    affiliation_city=None, affiliation_id=None, affiliation_name=None,
                    affiliation_country=None,
                    subject_area_name=None, subject_area_code=None, subject_area_abbrev=None,
                    publicationName=None, source_id=None, citedby_count=None,
                    subtype=None, dc_title=None, prism_doi=None,
                    dc_identifier=None, dc_publisher=None
                ))

            return rows

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {file_path}: {e}")
            return [Row(
                filename=os.path.basename(file_path),
                status="JSON Decode Error",
                srctype=None, eid=None, prism_coverDate=None,
                prism_aggregationType=None, subtypeDescription=None,
                affiliation_city=None, affiliation_id=None, affiliation_name=None,
                affiliation_country=None,
                subject_area_name=None, subject_area_code=None, subject_area_abbrev=None,
                publicationName=None, source_id=None, citedby_count=None,
                subtype=None, dc_title=None, prism_doi=None,
                dc_identifier=None, dc_publisher=None
            )]
        except Exception as e:
            print(f"Unexpected error in file {file_path}: {e}")
            return [Row(
                filename=os.path.basename(file_path),
                status="Unexpected Error",
                srctype=None, eid=None, prism_coverDate=None,
                prism_aggregationType=None, subtypeDescription=None,
                affiliation_city=None, affiliation_id=None, affiliation_name=None,
                affiliation_country=None,
                subject_area_name=None, subject_area_code=None, subject_area_abbrev=None,
                publicationName=None, source_id=None, citedby_count=None,
                subtype=None, dc_title=None, prism_doi=None,
                dc_identifier=None, dc_publisher=None
            )]

# Collect all files in the folder
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(f"Total files: {len(files)}")

# Process files
rdd = spark.sparkContext.parallelize(files).flatMap(process_file)

# Define schema dynamically
columns = [
    "filename", "status", "srctype", "eid", "prism_coverDate", "prism_aggregationType", "subtypeDescription",
    "affiliation_city", "affiliation_id", "affiliation_name", "affiliation_country",
    "subject_area_name", "subject_area_code", "subject_area_abbrev",
    "publicationName", "source_id", "citedby_count", "subtype", "dc_title", "prism_doi", "dc_identifier", "dc_publisher"
]

# Create DataFrame
df = spark.createDataFrame(rdd)

# Show a sample of the DataFrame
df = df.orderBy("filename")

# Write to CSV
df.write.csv(output_csv, header=True, mode="overwrite")

print(f"Data successfully written to {output_csv}")


Total files: 2890
Data successfully written to /content/2023_coredata


In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# สร้าง SparkSession
spark = SparkSession.builder.appName("FilterDC_Title").getOrCreate()

# โหลดข้อมูลจากไฟล์ CSV
file_path = "/content/2023_coredata/part-00000-64fc6d7b-b3a3-4502-ae65-2129dfc22f83-c000.csv"  # เปลี่ยนตาม path ของไฟล์จริง
df = spark.read.csv(file_path, header=True, inferSchema=True)

# ลบคอลัมน์ที่ไม่ต้องการ
columns_to_drop = ["affiliation_city", "affiliation_id", "affiliation_name"]
df = df.drop(*columns_to_drop)

# เลือก affiliation_country ที่ต่างกันในแต่ละ dc_title
result_df = df.dropDuplicates(["dc_title", "affiliation_country"])

result_df = result_df.orderBy("filename")

# จัดเรียงคอลัมน์ใหม่
result_df = result_df.select("filename", "dc_title", "affiliation_country", "srctype", "eid", "prism_coverDate", "prism_aggregationType", "subtypeDescription",
    "subject_area_name", "subject_area_code", "subject_area_abbrev",
    "publicationName", "source_id", "citedby_count", "subtype", "prism_doi", "dc_identifier", "dc_publisher")

# หากต้องการบันทึกผลลัพธ์ลงไฟล์
output_path = "/content/2023_data_clean"
result_df.write.csv(output_path, header=True)
